# Notebook to calculate Inter Annotator Agreement

### Import the libraries

In [19]:
import sys
import pandas as pd
import collections 
import os
import numpy as np
from itertools import chain
from itertools import combinations
sys.path.insert(0, '..')
from src.experiment_utils.helper_classes import token, span, repository
from src.d02_corpus_statistics.corpus import Corpus
from src.d03_inter_annotator_agreement.inter_annotator_agremment import Inter_Annotator_Agreement, _get_score_article
from definitions import df_annotation_marker
from src.d03_inter_annotator_agreement.inter_annotator_agremment import row_to_span_list, keep_valid_anotations


from definitions import ROOT_DIR


## Small Tutorial

Load the dataframe stat_df

In [20]:
    
dataframe_dir = os.path.join(ROOT_DIR,'data/02_processed_to_dataframe', 'preprocessed_dataframe.pkl')
stat_df = pd.read_pickle(dataframe_dir)
stat_df.head()

,Policy,Text,Tokens,Article_State,Finished_Annotators,Curation,Alisha,Fride,Onerva,Fabian,Lynn,Sebastian,Joel
EU_32018R1999_Title_0_Chapter_7_Section_3_Article_43,,article 43\r\nexercise of the delegation\r\n1....,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Alisha, Fride]",[span id:CUR0 annotator:Curation layer:Instrum...,[span id:A1 annotator:Alisha layer:Instrumentt...,[span id:B1 annotator:Fride layer:Policydesign...,,,,,
EU_32019R0631_Title_0_Chapter_0_Section_0_Article_12,,article 12\r\nreal-world co2 emissions and fue...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Onerva, Fabian]",[span id:CUR36 annotator:Curation layer:Instru...,,,[span id:C1 annotator:Onerva layer:Instrumentt...,[span id:D1 annotator:Fabian layer:Policydesig...,,,
EU_32018L2001_Title_0_Chapter_0_Section_0_Article_11,,article 11\r\njoint projects between member st...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[span id:CUR116 annotator:Curation layer:Instr...,,[span id:B28 annotator:Fride layer:Instrumentt...,[span id:C58 annotator:Onerva layer:Instrument...,,,,
EU_32018R1999_Title_0_Chapter_7_Section_3_Article_56,,article 56\r\namendments to directive (eu) 201...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Alisha, Fride]",[span id:CUR202 annotator:Curation layer:Polic...,[span id:A38 annotator:Alisha layer:Policydesi...,[span id:B129 annotator:Fride layer:Policydesi...,,,,,
EU_32018L2001_Title_0_Chapter_0_Section_0_Article_03,,article 3\r\nbinding overall union target for ...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva, Fabian]",[span id:CUR211 annotator:Curation layer:Instr...,,[span id:B138 annotator:Fride layer:Instrument...,[span id:C165 annotator:Onerva layer:Instrumen...,[span id:D27 annotator:Fabian layer:Instrument...,,,


First create a object of class Inter annotator agreement. The constructor takes a stat_df as input, has a optional argument DEBUG where only the first 10 articles are taken to test different functions

In [21]:
test_evaluator = Inter_Annotator_Agreement(stat_df, front_and_whereas = False)
test_evaluator_debug = Inter_Annotator_Agreement(stat_df, DEBUG = True)


In [22]:
test_evaluator.df.shape

(412, 13)

Inter_Annotator_Agreement is a child class of the Corpus class, so all methods of the Corpus class are available

In [23]:
test_dir = repository(policy = 'EU_32008R1099')
test_evaluator.get_span_list(conditional_rep = test_dir, columns = 'annotators', item = 'tag', value =  'Tech_LowCarbon')

[span id:C3882 annotator:Onerva layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon start:18 stop:25 text:nuclear,
 span id:C3883 annotator:Onerva layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon start:95 stop:109 text:nuclear energy,
 span id:C3884 annotator:Onerva layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon start:151 stop:158 text:nuclear,
 span id:C5161 annotator:Onerva layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon start:125 stop:141 text:renewable energy,
 span id:C5162 annotator:Onerva layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon start:393 stop:409 text:renewable energy,
 span id:C5163 annotator:Onerva layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon start:499 stop:515 text:renewable energy,
 span id:C5164 annotator:Onerva layer:Technologyand

To calculate the inter annonator agreement, there are two options


## Append the score to dataframe

This method appends the inter-annotator agreement for each article which at least two valid annoations based on a set of inter-annotator agreement measures. The scores are calculated in parallel, this is the recommended method for computationally intensive scores.

First, we only consider the articles where the curation is finished and at least two annotators are present:

In [24]:
test_evaluator.keep_only_finished_articles()

In [25]:
#scoring_metrics = ['f1_exact', 'f1_tokenwise', 'f1_partial', 'f1_heuristic']

In [26]:
scoring_metrics = ['f1_exact', 'f1_tokenwise', 'f1_partial', 'f1_heuristic', 'pygamma']

In [27]:
test_evaluator.append_total_score_per_article(scoring_metrics)

100%|██████████| 412/412 [08:43<00:00,  1.27s/it]   


There is also a normal implementation which uses parallel

In [30]:
#test_evaluator.append_total_score_per_article_parallel(scoring_metrics)

Checking out the dataframe now:

In [31]:
test_evaluator.df.head()

,Policy,Text,Tokens,Article_State,Finished_Annotators,Curation,Alisha,Fride,Onerva,Fabian,Lynn,Sebastian,Joel,f1_exact_score,f1_tokenwise_score,f1_partial_score,f1_heuristic_score,pygamma_score
EU_32018R1999_Title_0_Chapter_7_Section_3_Article_43,,article 43\r\nexercise of the delegation\r\n1....,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Alisha, Fride]",[span id:CUR0 annotator:Curation layer:Instrum...,[span id:A1 annotator:Alisha layer:Instrumentt...,[span id:B1 annotator:Fride layer:Policydesign...,,,,,,0.218750,0.226574,0.281250,0.281250,0.233182
EU_32019R0631_Title_0_Chapter_0_Section_0_Article_12,,article 12\r\nreal-world co2 emissions and fue...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Onerva, Fabian]",[span id:CUR36 annotator:Curation layer:Instru...,,,[span id:C1 annotator:Onerva layer:Instrumentt...,[span id:D1 annotator:Fabian layer:Policydesig...,,,,0.289157,0.263793,0.409639,0.421603,0.301722
EU_32018L2001_Title_0_Chapter_0_Section_0_Article_11,,article 11\r\njoint projects between member st...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[span id:CUR116 annotator:Curation layer:Instr...,,[span id:B28 annotator:Fride layer:Instrumentt...,[span id:C58 annotator:Onerva layer:Instrument...,,,,,0.567308,0.512946,0.653846,0.653846,0.540192
EU_32018R1999_Title_0_Chapter_7_Section_3_Article_56,,article 56\r\namendments to directive (eu) 201...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Alisha, Fride]",[span id:CUR202 annotator:Curation layer:Polic...,[span id:A38 annotator:Alisha layer:Policydesi...,[span id:B129 annotator:Fride layer:Policydesi...,,,,,,0.736842,0.583333,0.736842,0.736842,0.577485
EU_32018L2001_Title_0_Chapter_0_Section_0_Article_03,,article 3\r\nbinding overall union target for ...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva, Fabian]",[span id:CUR211 annotator:Curation layer:Instr...,,[span id:B138 annotator:Fride layer:Instrument...,[span id:C165 annotator:Onerva layer:Instrumen...,[span id:D27 annotator:Fabian layer:Instrument...,,,,0.420198,0.410204,0.532749,0.544437,0.408946


### Get total score

test_evaluator.get_total_score_df(weight_by = 'Spans')To retrieve the total score of the corpus, use get_total_score_df() on a dataframes where the scores for individual articles have been calculated.

In [32]:
test_evaluator.get_total_score_df(weight_by = 'no_weighting')

{'f1_exact_score': 0.40028724799007476,
 'f1_tokenwise_score': 0.397687590370067,
 'f1_partial_score': 0.5159440267362017,
 'f1_heuristic_score': 0.5273362748620107,
 'pygamma_score': 0.38959430074850043}

In [33]:
test_evaluator.get_total_score_df(weight_by = 'Tokens')

{'f1_exact_score': 0.37492081552891304,
 'f1_tokenwise_score': 0.35240598751298724,
 'f1_partial_score': 0.4739162741735001,
 'f1_heuristic_score': 0.48679580434940306,
 'pygamma_score': 0.36993661738021266}

In [34]:
test_evaluator.get_total_score_df(weight_by = 'Spans')

{'f1_exact_score': 0.3874058515882508,
 'f1_tokenwise_score': 0.36379438736998276,
 'f1_partial_score': 0.4885814966811494,
 'f1_heuristic_score': 0.5028562323485702,
 'pygamma_score': 0.38221583739383}

if only specific scores are required:

In [35]:
test_evaluator.get_total_score_df('f1_exact_score', weight_by = 'no_weighting')

{'f1_exact_score': 0.40028724799007476}

or

In [36]:
test_evaluator.get_total_score_df(['f1_exact_score', 'f1_tokenwise_score'], weight_by = 'Spans')

{'f1_exact_score': 0.3874058515882508,
 'f1_tokenwise_score': 0.36379438736998276}

### Get total score per annotator

In [37]:
annotators = ['Onerva', 'Alisha', 'Fabian', 'Fride']
for ann in annotators:
    print('annotator: ', ann)
    print(test_evaluator.get_total_score_df(annotator = ann, weight_by = 'Spans'))
    print('----------------')

annotator:  Onerva
{'f1_exact_score': 0.4381654829355101, 'f1_tokenwise_score': 0.4090364908255155, 'f1_partial_score': 0.5339943394957395, 'f1_heuristic_score': 0.5467190944483692, 'pygamma_score': 0.4265219202853624}
----------------
annotator:  Alisha
{'f1_exact_score': 0.32323742866301736, 'f1_tokenwise_score': 0.30806281985414746, 'f1_partial_score': 0.42750169278836747, 'f1_heuristic_score': 0.44385587315930597, 'pygamma_score': 0.3266427212696107}
----------------
annotator:  Fabian
{'f1_exact_score': 0.3071820274746298, 'f1_tokenwise_score': 0.2747252372274946, 'f1_partial_score': 0.38820915429061104, 'f1_heuristic_score': 0.3962207997917874, 'pygamma_score': 0.29084655831618844}
----------------
annotator:  Fride
{'f1_exact_score': 0.4427467808588547, 'f1_tokenwise_score': 0.4214078450791374, 'f1_partial_score': 0.5584121507299492, 'f1_heuristic_score': 0.576894166530852, 'pygamma_score': 0.4401334871489934}
----------------


In [38]:
test_evaluator.get_score_annotator('Fride', ['f1_exact_score', 'f1_tokenwise_score'])

{'f1_exact_score': 0.44287267910542477,
 'f1_tokenwise_score': 0.42132905468298554}

### Rank articles by score

In [39]:
test_evaluator.df.sort_values(by=['f1_heuristic_score'])

,Policy,Text,Tokens,Article_State,Finished_Annotators,Curation,Alisha,Fride,Onerva,Fabian,Lynn,Sebastian,Joel,f1_exact_score,f1_tokenwise_score,f1_partial_score,f1_heuristic_score,pygamma_score
EU_32012L0027_Title_0_Chapter_4_Section_0_Article_21,,article 21\r\nconversion factors\r\nfor the pu...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[span id:CUR7809 annotator:Curation layer:Tech...,,[span id:B4371 annotator:Fride layer:Technolog...,[span id:C4487 annotator:Onerva layer:Technolo...,,,,,0.0,0.000000,0.0,0.0,-0.187198
EU_32019R0631_Title_0_Chapter_0_Section_0_Article_18,,article 18\r\nrepeal\r\nregulations (ec) no 44...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Onerva, Fabian]",[span id:CUR13524 annotator:Curation layer:Pol...,,,[span id:C6758 annotator:Onerva layer:Policyde...,[span id:D3340 annotator:Fabian layer:Policyde...,,,,0.0,0.000000,0.0,0.0,-0.140600
EU_32008R1099_Title_0_Chapter_0_Section_0_Article_08,,article 8\r\nannual nuclear statistics\r\nthe ...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Alisha, Fride]",[span id:CUR6743 annotator:Curation layer:Inst...,[span id:A2293 annotator:Alisha layer:Instrume...,[span id:B3594 annotator:Fride layer:Policydes...,[span id:C3873 annotator:Onerva layer:Policyde...,,,,,0.0,0.000000,0.0,0.0,-0.095282
EU_32008R1099_Title_0_Chapter_0_Section_0_Article_05,,article 5\r\ntransmission and dissemination\r\...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Alisha, Fride]",[span id:CUR15065 annotator:Curation layer:Ins...,[span id:A5801 annotator:Alisha layer:Instrume...,[span id:B8089 annotator:Fride layer:Instrumen...,[span id:C7141 annotator:Onerva layer:Policyde...,,[],,,0.0,0.000000,0.0,0.0,-0.037073
EU_32006L0066_Title_0_Chapter_0_Section_0_Article_29,,article 29\r\nentry into force\r\nthis directi...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fride, Onerva]",[span id:CUR10714 annotator:Curation layer:Pol...,,[span id:B6443 annotator:Fride layer:Policydes...,[span id:C5728 annotator:Onerva layer:Policyde...,,[],,,0.0,0.000000,0.0,0.0,-0.763486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EU_32009R0079_Title_0_Chapter_0_Section_0_Article_14,,article 14\r\namendments to directive 2007/46/...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Fabian, Onerva]",[span id:CUR12469 annotator:Curation layer:Pol...,,,[span id:C6392 annotator:Onerva layer:Policyde...,[span id:D2925 annotator:Fabian layer:Policyde...,,,,1.0,1.000000,1.0,1.0,1.000000
EU_32019L0944_Title_0_Chapter_7_Section_5_Article_73,,article 73\r\nentry into force\r\nthis directi...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Alisha, Fabian]",[span id:CUR12501 annotator:Curation layer:Pol...,[span id:A4492 annotator:Alisha layer:Policyde...,,[],[span id:D2942 annotator:Fabian layer:Policyde...,,,,0.0,0.846465,1.0,1.0,0.956143
EU_32018R1999_Title_0_Chapter_7_Section_3_Article_54,,article 54\r\namendments to directive 2012/27/...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Alisha, Fride]",[span id:CUR9806 annotator:Curation layer:Poli...,[span id:A3223 annotator:Alisha layer:Policyde...,[span id:B5863 annotator:Fride layer:Policydes...,,,,,,1.0,1.000000,1.0,1.0,1.000000
EU_32008R1099_Title_0_Chapter_0_Section_0_Article_12,,article 12\r\nentry into force\r\nthis regulat...,"[start:0 stop:7 text:article tag_count:0, star...",CURATION_FINISHED,"[Alisha, Fride]",[span id:CUR384 annotator:Curation layer:Polic...,[span id:A100 annotator:Alisha layer:Policydes...,[span id:B243 annotator:Fride layer:Policydesi...,[span id:C268 annotator:Onerva layer:Policydes...,,,,,0.5,0.857143,1.0,1.0,0.960483


## Get total score based on a spanlist

The inter annotator agreement score can be also calculated from a spanlist. For all the spans present, it calculates the inter agreement scores for alle the articls with at least two valid annoations. Can be used to caluclate simmilarity to curation.

In [41]:
test_dir = repository.from_repository_name('EU_32008R1099_Title_0_Chapter_0_Section_0_Article_12')
span_list = test_evaluator.get_span_list(test_dir, ['Onerva', 'Fride'])

In [42]:
span_list


[span id:C268 annotator:Onerva layer:Policydesigncharacteristics type:Time tag:Time_InEffect start:52 stop:156 text:enter into force on the 20th day following its publication in the official journal of the european union,
 span id:C269 annotator:Onerva layer:Policydesigncharacteristics type:Actor tag:Addressee_default start:239 stop:252 text:member states,
 span id:B243 annotator:Fride layer:Policydesigncharacteristics type:Time tag:Time_InEffect start:76 stop:134 text:20th day following its publication in the official journal,
 span id:B244 annotator:Fride layer:Policydesigncharacteristics type:Actor tag:Addressee_default start:239 stop:252 text:member states]

In [43]:
test_evaluator.get_score_spanlist(span_list, 'pygamma')

0.9476514351793441

Or use this function to get scores in specific categories:

In [63]:
test_dir = repository.from_repository_name('EU_32008R1099_Title_0_Chapter_0_Section_0_Article_05')
span_list = test_evaluator.get_span_list(test_dir, columns = ['Alisha', 'Fride'], item = 'layer', value = 'Policydesigncharacteristics')
span_list

[span id:A5803 annotator:Alisha layer:Policydesigncharacteristics type:Actor tag:Addressee_monitored start:48 stop:61 text:member states,
 span id:A5804 annotator:Alisha layer:Policydesigncharacteristics type:Actor tag:Authority_monitoring start:84 stop:94 text:commission,
 span id:A5805 annotator:Alisha layer:Policydesigncharacteristics type:Actor tag:Authority_monitoring start:96 stop:104 text:eurostat,
 span id:A5806 annotator:Alisha layer:Policydesigncharacteristics type:Compliance tag:Form_monitoring start:110 stop:129 text:national statistics,
 span id:A5807 annotator:Alisha layer:Policydesigncharacteristics type:Compliance tag:Form_monitoring start:366 stop:385 text:national statistics,
 span id:A5808 annotator:Alisha layer:Policydesigncharacteristics type:Actor tag:Addressee_monitored start:528 stop:540 text:member state,
 span id:A5809 annotator:Alisha layer:Policydesigncharacteristics type:Reversibility tag:Reversibility_policy start:553 stop:578 text:exemptions or derogation

In [64]:
test_evaluator.get_score_spanlist(span_list, 'f1_heuristic')

0.0

## Check closeness to curation

To check the agreement with the curation of all annotators, we simply create a spanlist for each annotator containing all his spans and the ones from the curation
Since this is based on a big list instead of a dataframe, the computation is very slow

In [66]:
test_evaluator.append_score_to_curation(scoring_metrics)

100%|██████████| 412/412 [01:10<00:00,  5.82it/s]


### Get individual score

In [73]:
test_evaluator.get_to_curation_score(weight_by = 'no_weighting')

{'Onerva': {'f1_partial': 0.6732862244002309,
  'f1_heuristic': 0.6176183342356287,
  'f1_tokenwise': 0.7160416942508684,
  'pygamma': 0.7418035160958896,
  'f1_exact': 0.6014000379564838},
 'Fride': {'f1_partial': 0.6829956559202468,
  'f1_heuristic': 0.6406458629220034,
  'f1_tokenwise': 0.7364141686616807,
  'pygamma': 0.7520213456882968,
  'f1_exact': 0.6266985333457066},
 'Fabian': {'f1_partial': 0.43231750058721435,
  'f1_heuristic': 0.37760504426835484,
  'f1_tokenwise': 0.49015694287418154,
  'pygamma': 0.5092491205438597,
  'f1_exact': 0.3779958044165836},
 'Alisha': {'f1_partial': 0.574676845271569,
  'f1_heuristic': 0.5344264074636121,
  'f1_tokenwise': 0.6370336784394007,
  'pygamma': 0.6622106992758143,
  'f1_exact': 0.5360664298052316}}

In [74]:
test_evaluator.get_to_curation_score(weight_by = 'Tokens')

{'Onerva': {'f1_partial': 0.6497229248404979,
  'f1_heuristic': 0.588104303406712,
  'f1_tokenwise': 0.6926120329988612,
  'pygamma': 0.7155316902447743,
  'f1_exact': 0.5975570197174063},
 'Fride': {'f1_partial': 0.6807736554301652,
  'f1_heuristic': 0.6241358009693133,
  'f1_tokenwise': 0.7273136810436603,
  'pygamma': 0.7406803749512647,
  'f1_exact': 0.6321752810813952},
 'Fabian': {'f1_partial': 0.3884944280696229,
  'f1_heuristic': 0.3137100596814219,
  'f1_tokenwise': 0.42722291795144346,
  'pygamma': 0.44587542344065123,
  'f1_exact': 0.3263799326874162},
 'Alisha': {'f1_partial': 0.5475251978127618,
  'f1_heuristic': 0.49672538277097095,
  'f1_tokenwise': 0.6053944023319054,
  'pygamma': 0.630782505115647,
  'f1_exact': 0.5095565539915253}}

In [75]:
test_evaluator.get_to_curation_score(weight_by = 'Spans')

{'Onerva': {'f1_partial': 0.6487655252174788,
  'f1_heuristic': 0.5876631656187074,
  'f1_tokenwise': 0.6919092874742293,
  'pygamma': 0.7151208327340163,
  'f1_exact': 0.597720841520636},
 'Fride': {'f1_partial': 0.6829841040462425,
  'f1_heuristic': 0.6268153336863214,
  'f1_tokenwise': 0.7295881502890171,
  'pygamma': 0.7434392410812806,
  'f1_exact': 0.6358002337794381},
 'Fabian': {'f1_partial': 0.3859744866634834,
  'f1_heuristic': 0.30882989118106857,
  'f1_tokenwise': 0.4219933146872233,
  'pygamma': 0.4412883241131821,
  'f1_exact': 0.3218379490907672},
 'Alisha': {'f1_partial': 0.5462982718156604,
  'f1_heuristic': 0.4947175134484899,
  'f1_tokenwise': 0.6046511627906977,
  'pygamma': 0.6314991627071533,
  'f1_exact': 0.5082830865188923}}

### Get Total score

In [76]:
test_evaluator.get_to_curation_score_total(weight_by = 'no_weighting')

{'f1_partial': 0.5988896876960187,
 'f1_heuristic': 0.5511128608780697,
 'f1_tokenwise': 0.65271203543617,
 'pygamma': 0.6738591198644103,
 'f1_exact': 0.5435373507496893}

In [77]:
test_evaluator.get_to_curation_score_total(weight_by = 'Tokens')

{'f1_partial': 0.5720544815512948,
 'f1_heuristic': 0.5114731630019173,
 'f1_tokenwise': 0.6185247079726235,
 'pygamma': 0.6385253916986301,
 'f1_exact': 0.5221356685927355}

In [78]:
test_evaluator.get_to_curation_score_total(weight_by = 'Spans')

{'f1_partial': 0.5742673921447767,
 'f1_heuristic': 0.5134787912374995,
 'f1_tokenwise': 0.6211169455066855,
 'pygamma': 0.6417420284651623,
 'f1_exact': 0.5248772371194387}

# Check scores in different categories

In [79]:
layers = ['Technologyandapplicationspecificity', 'Policydesigncharacteristics', 'Instrumenttypes' ]
repo = repository()

for l in layers:
    span_list_layer = test_evaluator.get_span_list(repo, columns = 'annotators', item = 'layer', value = l)
    score = test_evaluator.get_score_spanlist(span_list_layer, 'f1_heuristic')
    print('layer: ', l, ', score: ', score)
    

AttributeError: 'numpy.float64' object has no attribute 'rep'

In [ ]:
all_tags = 